In [20]:
import zipline
from zipline.api import future_symbol, \
    set_commission, set_slippage, schedule_function, date_rules, \
    time_rules, continuous_future, order_target
from zipline.finance.commission import PerTrade, PerContract
from zipline.finance.slippage import VolumeShareSlippage, \
    FixedSlippage, VolatilityVolumeShare

from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import pyfolio as pf
import pandas as pd
import numpy as np

%matplotlib inline

In [21]:
# These lines below are for dynamic text reporting
from IPython.display import display
import ipywidgets as widgets
out = widgets.HTML()
display(out)

HTML(value='')

In [22]:
# Model settings:
starting_portfolio = 50000000
risk_factor = 0.0015
stop_distance = 3
breakout_window = 50
vola_window = 40
enable_commission = True
enable_slippage = True

In [23]:
def report_result(context, data):
    context.months += 1
    today = zipline.api.get_datetie().date()
    
    # Calculate annualized return so far
    ann_ret = np.power(context.portfolio_value / starting_portfolio, 12 / context.months) - 1
    
    # Update the text
    out.value = """{} We have traded <b>{}<b> months
    and the annualized return is <b>{:.2%}<b>""".format(today, context.months, ann_ret)

In [24]:
def roll_futures(context, data):
    open_orders = zipline.api.get_open_orders()
    
    for held_contract in context.portfolio.positions:
        # don't roll positions that are set to change by core logic
        if held_contract in open_orders:
            continue
            
        # Save some time by only checking rolls for contracts stopping trading in the next days
        days_to_auto_close = (held_contract.auto_close_date.date() - data.current_session.date()).days
        if days_to_auto.close > 5:
            continue
            
        # Make a continuation
        continuation = continuous_future(held_contract.root_symbol, 
                                         offset=0,
                                         roll='volume',
                                         adjustment='mul' 
                                        )
        
        # Get the current contract of the continuation
        continuation_contract = data.current(continuation, 'contract')
        
        if continuation_contract != held_contract:
            # Check how many contracts we hold
            pos_size = context.portfolio.positions[held_contract].amount
            
            # Close current position
            order_target(held_contract, 0)
            
            # Open new position
            order_target(continuation_contract, pos_size)

In [25]:
def position_size(portfolio_value, std, point_value):
    target_variation = portfolio_value * risk_factor
    contract_variation = std * point_value
    contracts = target_variation / contract_variation
    return int(np.nan_to_num(contracts))

In [26]:
def initialize(context):
    # Cost settings
    if enable_commision:
        comm_model = PerContract(cost=0.85, exchange_fee=1.5)
    else:
        comm_model = PerTrade(cost=0.0)
    
    set_commission(us_futures=comm_model)
    
    if enable_slippage:
        slippage_model = VolatilityVolumeShare(volume_limit=0.2)
    else:
        slippage_model = FixedSliappage(spread=0.0)
        
    set_slippage(us_futures=slippage_model)
    
    # Markets to trade
    currencies = ['AD', 'BP', 'CD', 'CU', 'DX', 'JY', 'NE', 'SF']
    agricultural = ['BL', '_C', 'CT', 'FC', 'KC', 'LR', 'LS', '_O', '_S', 'SB', 'SM', '_W']
    nonagricultural = ['CL', 'GC', 'HG', 'HO', 'LG', 'NG', 'PA', 'PL', 'RB', 'SI']
    equities = ['ES', 'NK', 'NQ', 'TW', 'VX', 'YM']
    rates = ['ED', 'FV', 'TU', 'TY', 'US']
    
    # Make a list of all the markets
    markets = currencies + agricultural + nonagricultural + equities + rates
    
    # Make a list of all continuations
    context.universe = [
        continuous_future(market, offset=0, roll='volume', adjustment='mul')
        for market in markets
                       ]
    # We'll use this to keep trak of best position reading, used to calc. stop points:
    context.highest_in_position = {market: 0 for market in markets}
    context.lowest_in_position = {market: 0 for market in markets}
    
    # Schedule the daily trading
    schedule_function(daily_trade, date_rules.every_day(), time_rules.market_close())
    
    # This is used for progress output during the backtest
    context.months = 0
    
    # Schedule monthly report output
    schedule_function(func=report_result, 
                      date_rule=date_rules.month_start(), 
                      time_rule=time_rules.market_open()
                     )

In [ ]:
def analyze(context, data):
    # Get continuation data
    hist = data.history(context.universe, 
                        fields=['close', 'volume'], 
                        frequency='1d', 
                        bar_count=250
                       )